# Datenaufbereitung und Visualisierung Modularbeit

Author - Maximilian Hoffmann 2023

**Table of Contents**

- [Introduction](#introduction)
- [Hypothesis](#hypothesis)
- [Data](#data)
- [Data Preparation](#data-preparation)
- [Data Visualization](#data-visualization)
- [Data Analysis](#data-analysis)
- [Anonymization](#anonymization)
- [Conclusion](#conclusion)
- [References](#references)


## Introduction

Für den Landtagswahlkampf Artikel soll ein Thema gefunden werden, welches relevanz für die Wähler und somit auch den Wahlkampf selbst hat. Dazu soll ein Datensatz gefunden werden, welcher die Relevanz des Themas belegt. Dieser Datensatz soll dann in einer Visualisierung dargestellt werden.

## Hypothesis

Da durch die Energiekostenexplosion im Jahr 2021 [<a id="reref1">[1](#ref1)], die Wähler sich mehr für das Thema Energie interessieren und für eine Energiewende einsetzen. Die Hypothese ist, dass es möglich ist allein durch Balkonkraftwerke die Energieversorgung eines Haushaltes zu decken und somit die Energiekosten zu senken. Diese Hypothese soll durch die Visualisierung belegt werden und Zeigen das eine Lockerung der Gesetze für Balkonkraftwerke sinnvoll ist. Das Ziel Klimaneutral zu werden würde so gefördert werden.

Weiterführend soll die Visualisierung zeigen, das durch die setigenden Energiekosten in den letzten Jahren Balkonkraftwerke zukünftig noch mehr an Bedeutung gewinnen werden. Ebenfalls soll gezeigt werden dass der Umstieg von fossilen Brennstoffen auf erneuerbare Energien sinnvoll ist und die abhängigkeit Deutschlands von anderen Ländern verringert.

Die Wahl des Balkonkraftwerks entstand da es sich um eine einfache und günstige Möglichkeit handelt, erneuerbare Energien zu nutzen. Hinzu ist es jedem Haushalt möglich, erneuerbare Energien zu nutzen und somit einen Beitrag zum Klimaschutz zu leisten. 

## Data


Die hierzu genutzten Datensätze des Datenarchivs [Open Data Bayern](https://opendata.bayern) sind:

- [Anteil Photovoltaik am Stromverbrauch (Gem.)](https://opendata.bayern/datasets/db272dfa-7dfb-45fb-9459-01840a3ce992?locale=de) - Bayerisches Landesamt für Umwelt
- [Anteil erneuerbarer Energien am Stromverbrauch (Gem.)](https://opendata.bayern/datasets/9cebb88d-3877-4afe-a24c-3f24757f8257?locale=de) - Bayerisches Landesamt für Umwelt

Zudem wurden Daten aus anderen Quellen genutzt:

- [Destatis](https://www.destatis.de/)
	- [Preisentwicklungen](https://www.destatis.de/DE/Presse/Pressemitteilungen/2022/03/PD22_N016_61.html)
- [Bayerisches Staatsministerium für Wirtschaft, Landesentwicklung und Energie](https://www.stmwi.bayern.de)
	- [bayerische Energiebilanz 2020](https://www.stmwi.bayern.de/energie/energiedaten/energiebilanz-2020/)

Selbst erhobene Daten aus Balkonkraftwerk-Testanlage in München (siehe [Anonymization](#anonymization)):

- Die CSV-Datei wurde über die Messung eines zwischengeschalteten Shelly-Plus-1PM (WLAN-Schalter mit Strommessfunktion) erhoben.

## Data Preparation

## Data Visualization

Für die Datenvisualisierung werden folgende Bibliotheken genutzt:


<!-- TODO: DELETE BELOW -->

- [Matplotlib](https://matplotlib.org/)
- [Pandas](https://pandas.pydata.org/)
- [Numpy](https://numpy.org/)
- [Seaborn](https://seaborn.pydata.org/)
- [Plotly](https://plotly.com/python/)
- [Folium](https://python-visualization.github.io/folium/)
- [Geopandas](https://geopandas.org/)
- [Geojson](https://geojson.org/)
- [Shapely](https://shapely.readthedocs.io/en/stable/manual.html)
- [Pyproj](https://pyproj4.github.io/pyproj/stable/)
- [Geopy](https://geopy.readthedocs.io/en/stable/)
- [Requests](https://docs.python-requests.org/en/master/)
- [Json](https://docs.python.org/3/library/json.html)
- [sklearn](https://scikit-learn.org/stable/)


In [1]:
# importing the required libraries


ModuleNotFoundError: No module named 'folium'

## Data Analysis

## Anonymization

## Conclusion

Die oben visualisierten Daten belegen, ...

## References

- [<a id="ref1">[1](#reref1)] : [Energiepreisentwicklung](https://www.destatis.de/DE/Presse/Pressemitteilungen/2022/03/PD22_N016_61.html)
- [<a id="ref2">[2](#reref2)] : [Open Data Bayern](https://opendata.bayern)